# Librería básica de álgebra lineal (notebook)
Este notebook contiene una versión simple de la librería `lac` en 4 celdas:
- Vector y funciones (lac.vector)
- Matrix y funciones (lac.matrix)
- Demo / pruebas

In [1]:
import math
from typing import List

class Vector:
    def __init__(self, componentes: List[float]):
        self.componentes = list(componentes)
    
    def __repr__(self):
        return f"Vector({self.componentes})"
    
    def __len__(self):
        return len(self.componentes)
    
    def __getitem__(self, i):
        return self.componentes[i]
    
    def __eq__(self, other):
        return isinstance(other, Vector) and self.componentes == other.componentes

    # operadores
    def __add__(self, otro):
        return Vector([a + b for a, b in zip(self.componentes, otro.componentes)])
    
    def __sub__(self, otro):
        return Vector([a - b for a, b in zip(self.componentes, otro.componentes)])
    
    # multiplicación por escalar (v * scalar)
    def __mul__(self, escalar):
        if isinstance(escalar, (int, float)):
            return Vector([a * escalar for a in self.componentes])
        raise NotImplementedError("Sólo soporte multiplicación por escalar")
    
    # soportar scalar * v
    def __rmul__(self, escalar):
        return self.__mul__(escalar)
    
    # magnitud
    def magnitude(self):
        return math.sqrt(sum(a*a for a in self.componentes))
    
    # producto punto
    def dot(self, otro):
        return sum(a * b for a, b in zip(self.componentes, otro.componentes))
    
    # distancia
    def distance(self, otro):
        return (self - otro).magnitude()


# ==== Funciones en lac.vector (simples wrappers) ====
def add_vectors(v1: Vector, v2: Vector) -> Vector:
    return v1 + v2

def subtract_vectors(v1: Vector, v2: Vector) -> Vector:
    return v1 - v2

def scale_vector(v: Vector, scalar: float) -> Vector:
    return v * scalar

def dot_vectors(v1: Vector, v2: Vector) -> float:
    return v1.dot(v2)

def magnitude_vector(v: Vector) -> float:
    return v.magnitude()


In [2]:
from copy import deepcopy
from typing import List

class Matrix:
    def __init__(self, datos: List[List[float]]):
        # validación mínima: todas filas misma longitud
        if any(len(fila) != len(datos[0]) for fila in datos):
            raise ValueError("Todas las filas deben tener la misma longitud")
        self.datos = deepcopy(datos)
    
    def __repr__(self):
        return f"Matrix({self.datos})"
    
    # propiedades num_rows, num_columns, shape
    @property
    def num_rows(self) -> int:
        return len(self.datos)
    
    @property
    def num_columns(self) -> int:
        return len(self.datos[0]) if self.datos else 0
    
    @property
    def shape(self):
        return (self.num_rows, self.num_columns)
    
    # Transpuesta (T)
    @property
    def T(self):
        return Matrix([[self.datos[r][c] for r in range(self.num_rows)] for c in range(self.num_columns)])
    
    # Traza (trace) — si no es cuadrada toma min(n,m)
    @property
    def trace(self):
        n = min(self.num_rows, self.num_columns)
        return sum(self.datos[i][i] for i in range(n))
    
    # Determinante (propiedad): implementamos recursivo (funcionará para tamaños pequeños del curso)
    @property
    def determinant(self):
        if self.num_rows != self.num_columns:
            raise ValueError("Determinante sólo definido para matrices cuadradas")
        n = self.num_rows
        # base cases
        if n == 1:
            return self.datos[0][0]
        if n == 2:
            a, b = self.datos[0]
            c, d = self.datos[1]
            return a * d - b * c
        # recursion (expansión por primera fila)
        det = 0
        for c in range(n):
            # construir minor sin fila 0 y columna c
            minor = [row[:c] + row[c+1:] for row in self.datos[1:]]
            sign = (-1) ** c
            det += sign * self.datos[0][c] * Matrix(minor).determinant
        return det
    
    # operadores __add__, __sub__, __mul__ (escalar)
    def __add__(self, otra):
        if self.shape != otra.shape:
            raise ValueError("Shapes must match for addition")
        return Matrix([[a + b for a, b in zip(f1, f2)] for f1, f2 in zip(self.datos, otra.datos)])
    
    def __sub__(self, otra):
        if self.shape != otra.shape:
            raise ValueError("Shapes must match for subtraction")
        return Matrix([[a - b for a, b in zip(f1, f2)] for f1, f2 in zip(self.datos, otra.datos)])
    
    # multiplicación por escalar (M * scalar)
    def __mul__(self, escalar):
        if isinstance(escalar, (int, float)):
            return Matrix([[a * escalar for a in fila] for fila in self.datos])
        raise NotImplementedError("Sólo se implementa multiplicación por escalar con __mul__")
    
    def __rmul__(self, escalar):
        return self.__mul__(escalar)
    
    def __eq__(self, otra):
        return isinstance(otra, Matrix) and self.datos == otra.datos

    # multiplicación matriz x matriz clásica (para métodos externos también)
    def matmul(self, otra):
        filasA, colsA = self.shape
        filasB, colsB = otra.shape
        if colsA != filasB:
            raise ValueError("Dimensiones incompatibles para multiplicación")
        resultado = []
        for i in range(filasA):
            nueva_fila = []
            for j in range(colsB):
                s = 0
                for k in range(colsA):
                    s += self.datos[i][k] * otra.datos[k][j]
                nueva_fila.append(s)
            resultado.append(nueva_fila)
        return Matrix(resultado)

# ==== Funciones en lac.matrix (wrappers) ====
def scale_matrix(m: Matrix, scalar: float) -> Matrix:
    return m * scalar

def add_matrices(m1: Matrix, m2: Matrix) -> Matrix:
    return m1 + m2

def subtract_matrices(m1: Matrix, m2: Matrix) -> Matrix:
    return m1 - m2

def vector_multiply(m: Matrix, v: Vector) -> Vector:
    # multiplicación matriz * vector (m filas x cols) * (vector len == cols) -> vector len = filas
    if m.num_columns != len(v):
        raise ValueError("Dimensiones incompatibles matriz x vector")
    resultado = []
    for fila in m.datos:
        resultado.append(sum(a * b for a, b in zip(fila, v.componentes)))
    return Vector(resultado)

def matrix_multiply(m1: Matrix, m2: Matrix) -> Matrix:
    return m1.matmul(m2)

In [3]:

print("=== VECTORES ===")
v1 = Vector([1,2,3])
v2 = Vector([4,5,6])
print("v1:", v1)
print("v2:", v2)
print("add_vectors:", add_vectors(v1, v2))
print("subtract_vectors:", subtract_vectors(v2, v1))
print("scale_vector v1*3:", scale_vector(v1, 3))
print("dot_vectors:", dot_vectors(v1, v2))
print("magnitude v1:", magnitude_vector(v1))

print("\n=== MATRICES ===")
m1 = Matrix([[1,2],[3,4]])
m2 = Matrix([[2,0],[1,2]])
print("m1:", m1)
print("m2:", m2)
print("shape m1:", m1.shape, "num_rows:", m1.num_rows, "num_columns:", m1.num_columns)
print("m1.T:", m1.T)
print("trace m1:", m1.trace)
print("determinant m1:", m1.determinant)
print("add_matrices:", add_matrices(m1, m2))
print("subtract_matrices:", subtract_matrices(m1, m2))
print("scale_matrix (m1*2):", scale_matrix(m1, 2))
print("matrix_multiply m1*m2:", matrix_multiply(m1, m2))

print("\nvector_multiply m2 * v (v len==cols):")
v3 = Vector([1,2])
print("v3:", v3)
print("m2 * v3:", vector_multiply(m2, v3))

# === Resumen de la evaluación (lo que implementamos) ===
puntajes = {
    "Vector (clase)": 30,
    "Funciones lac.vector": 20,
    "Matrix: num_cols/num_rows/shape": 5,
    "Matrix: T & trace": 10,
    "Matrix: determinant": 15,
    "__ dunder methods (ej. __add__ etc)": 3,
    "lac.matrix: scale/add/subtract": 5,
    "lac.matrix: vector_multiply": 5,
    "lac.matrix: matrix_multiply": 7
}
print("\nImplementado (suma de puntajes):", sum(puntajes.values()), "%")
print("Detalle implementado (todos los ítems anteriores están presentes).")

=== VECTORES ===
v1: Vector([1, 2, 3])
v2: Vector([4, 5, 6])
add_vectors: Vector([5, 7, 9])
subtract_vectors: Vector([3, 3, 3])
scale_vector v1*3: Vector([3, 6, 9])
dot_vectors: 32
magnitude v1: 3.7416573867739413

=== MATRICES ===
m1: Matrix([[1, 2], [3, 4]])
m2: Matrix([[2, 0], [1, 2]])
shape m1: (2, 2) num_rows: 2 num_columns: 2
m1.T: Matrix([[1, 3], [2, 4]])
trace m1: 5
determinant m1: -2
add_matrices: Matrix([[3, 2], [4, 6]])
subtract_matrices: Matrix([[-1, 2], [2, 2]])
scale_matrix (m1*2): Matrix([[2, 4], [6, 8]])
matrix_multiply m1*m2: Matrix([[4, 4], [10, 8]])

vector_multiply m2 * v (v len==cols):
v3: Vector([1, 2])
m2 * v3: Vector([2, 5])

Implementado (suma de puntajes): 100 %
Detalle implementado (todos los ítems anteriores están presentes).
